<a href="https://colab.research.google.com/github/kulharshit21/BeatBuddy/blob/main/dataset1_notebook_allmodel1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()  # This allows you to upload files interactively
file_path = list(uploaded.keys())[0]  # Gets the filename of the uploaded file



In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import scipy.signal as signal
from scipy import stats

# Bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data, axis=-1)

# Compute power for a specific frequency band
def band_power(data, band, fs):
    f, Pxx = signal.welch(data, fs, nperseg=1024)
    band_mask = (f >= band[0]) & (f <= band[1])
    return np.trapz(Pxx[band_mask], f[band_mask])

# List all .fdt files in the current directory
fdt_files = [f for f in os.listdir() if f.endswith('.fdt')]
print(f"Found {len(fdt_files)} .fdt files: {fdt_files}")

# Parameters
fs = 250  # Sampling rate (Hz)
channels = 32  # Hypothetical number of EEG channels
lowcut = 0.5  # Hz
highcut = 40  # Hz
delta_band = (0.5, 4)  # Delta band

# Process each file
for file_path in fdt_files:
    print(f"Processing file: {file_path}")
    data = np.fromfile(file_path, dtype=np.float32)

    # Reshape data into channels × time points
    time_points = data.size // channels
    reshaped_data = data.reshape((channels, time_points))

    # Apply bandpass filtering
    filtered_data = bandpass_filter(reshaped_data, lowcut, highcut, fs)

    # Example: Compute delta power for first channel
    delta_power = band_power(filtered_data[0], delta_band, fs)
    print(f"Delta power for {file_path}: {delta_power}")

    # Add further processing as needed


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import scipy.signal as signal
from scipy import stats

# Bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data, axis=-1)

# Compute power for a specific frequency band
def band_power(data, band, fs):
    f, Pxx = signal.welch(data, fs, nperseg=1024)
    band_mask = (f >= band[0]) & (f <= band[1])
    return np.trapz(Pxx[band_mask], f[band_mask])

# Plot EEG channels
def plot_eeg_channels(data, channels, time_range, fs, title="EEG Signal"):
    time = np.arange(time_range.start, time_range.stop) / fs  # Time in seconds
    plt.figure(figsize=(12, 8))
    for i, ch in enumerate(channels):
        plt.plot(time, data[ch, time_range] + i * 20, label=f"Channel {ch + 1}")  # Offset for clarity
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (µV)")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

# List all .fdt files in the current directory
fdt_files = [f for f in os.listdir() if f.endswith('.fdt')]
print(f"Found {len(fdt_files)} .fdt files: {fdt_files}")

# Parameters
fs = 250  # Sampling rate (Hz)
channels = 32  # Hypothetical number of EEG channels
lowcut = 0.5  # Hz
highcut = 40  # Hz
delta_band = (0.5, 4)  # Delta band
theta_band = (4, 8)  # Theta band
alpha_band = (8, 12)  # Alpha band

# Process each file
for file_path in fdt_files:
    print(f"Processing file: {file_path}")
    data = np.fromfile(file_path, dtype=np.float32)

    # Reshape data into channels × time points
    time_points = data.size // channels
    reshaped_data = data.reshape((channels, time_points))

    # Apply bandpass filtering
    filtered_data = bandpass_filter(reshaped_data, lowcut, highcut, fs)

    # Compute and plot spectrogram for the first channel
    f, t, Sxx = signal.spectrogram(filtered_data[0], fs)
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.title(f"Spectrogram of EEG (Channel 1) - {file_path}")
    plt.colorbar(label="Power/Frequency (dB/Hz)")
    plt.show()

    # Compute and plot Power Spectral Density (PSD) for the first channel
    f, Pxx_density = signal.welch(filtered_data[0], fs, nperseg=1024)
    plt.figure(figsize=(10, 6))
    plt.semilogy(f, Pxx_density)  # Use logarithmic scale for better visualization
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Power Spectral Density [V^2/Hz]')
    plt.title(f'Power Spectral Density (PSD) of EEG (Channel 1) - {file_path}')
    plt.grid(True)
    plt.show()

    # Compute band powers for two segments: REM and Non-REM (for demonstration)
    rem_data = filtered_data[0, 1000:2000]  # Example of REM data (first channel)
    non_rem_data = filtered_data[0, 2000:3000]  # Example of Non-REM data (first channel)

    rem_delta_power = band_power(rem_data, delta_band, fs)
    non_rem_delta_power = band_power(non_rem_data, delta_band, fs)

    # Perform t-test to compare powers
    t_stat, p_value = stats.ttest_ind([rem_delta_power], [non_rem_delta_power])

    print(f"T-statistic: {t_stat}, P-value: {p_value}")

    # Visualize first 4 channels and first 1000 time points
    channels_to_plot = [0, 1, 2, 3]
    time_range = slice(0, 1000)
    plot_eeg_channels(filtered_data, channels_to_plot, time_range, fs, title=f"EEG Signal - {file_path}")
